In [3]:
import os
from dotenv import load_dotenv
from typing import List
from langsmith import traceable 
from langchain_groq import ChatGroq
from utils import get_vector_db_retriever

# Load environment variables from .env file
load_dotenv()

c:\Users\Raghav Gupta\anaconda3\envs\langsmith_env\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


True

In [4]:
# --- Constants and Setup ---
MODEL_NAME = "llama-3.3-70b-versatile" 
RAG_SYSTEM_PROMPT = """You are an assistant for question-answering tasks. 
Use the following pieces of retrieved context to answer the question. 
If you don't know the answer, just say that you don't know. 
Use three sentences maximum and keep the answer concise.
"""
retriever = get_vector_db_retriever()

c:\Users\Raghav Gupta\OneDrive\Desktop\Langsmith-MAT496\utils.py:16: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding_model = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")


In [5]:
# --- Traced Functions (LCEL and Metadata Tweak) ---

@traceable(
    name="Retrieve Documents", 
    metadata={"retriever": "FAISS-HuggingFace"} # YOUR TWEAK: Adding context
)
def retrieve_documents(question: str):
    return retriever.invoke(question)

@traceable(
    name="Generate Response with Groq", 
    metadata={"model_name": MODEL_NAME} # YOUR TWEAK: Adding context
)
def generate_response(question: str, documents):
    formatted_docs = "\n\n".join(doc.page_content for doc in documents)
    messages = [
        {"role": "system", "content": RAG_SYSTEM_PROMPT},
        {"role": "user", "content": f"Context: {formatted_docs} \n\n Question: {question}"}
    ]
    chat = ChatGroq(model=MODEL_NAME)
    response = chat.invoke(messages)
    return response.content

@traceable(name="RAG Pipeline")
def langsmith_rag(question: str):
    documents = retrieve_documents(question)
    answer = generate_response(question, documents)
    return answer

In [6]:
# --- Main Execution ---
question = "How can I trace with the @traceable decorator?"
ai_answer = langsmith_rag(question)

print("--- Question ---")
print(question)
print("\n--- Answer ---")
print(ai_answer)

--- Question ---
How can I trace with the @traceable decorator?

--- Answer ---
You can trace a function by adding the @traceable decorator to it, which will automatically track its inputs, outputs, and errors in LangSmith. To add metadata, you can pass a 'metadata' dictionary to the @traceable decorator. Additionally, you can also add metadata at runtime using the 'langsmith_extra' parameter.
